In [1]:
import myk_models
import myk_train
import torch
import onnx
from onnx_tf.backend import prepare
from torch.autograd import Variable

2023-07-14 15:52:34.422313: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/macdonald/opt/miniconda3/envs/STMAE/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
device = myk_train.get_device()

cuda device not available/not selected


In [13]:
trained_model = myk_models.SimpleLSTM(hidden_size=32).to(device)
print(trained_model)

trained_model.load_state_dict(torch.load('models/LSTM_14-07-2023_13-14-30/model.ph', device))
dummy_input = Variable(torch.randn(1,32,1))
torch.onnx.export(trained_model, dummy_input, "model_test.onnx")

SimpleLSTM(
  (lstm): LSTM(1, 32, batch_first=True)
  (dense): Linear(in_features=32, out_features=1, bias=True)
)
================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [9]:
model = onnx.load('model_test.onnx')
tf_rep = prepare(model)

In [5]:
# import numpy as np
# import myk_data
# input_test = np.expand_dims(myk_data.load_wav_file('../data/guitar.wav', 44100), 0)

In [6]:
# tf_rep.save('saved_model.pb')

In [10]:
tf_rep.export_graph("model_test.pb")

ValueError: in user code:

    File "/Users/macdonald/Desktop/MagicKnobRep/onnx-tensorflow/onnx_tf/backend_tf_module.py", line 99, in __call__  *
        output_ops = self.backend._onnx_node_to_tensorflow_op(onnx_node,
    File "/Users/macdonald/Desktop/MagicKnobRep/onnx-tensorflow/onnx_tf/backend.py", line 347, in _onnx_node_to_tensorflow_op  *
        return handler.handle(node, tensor_dict=tensor_dict, strict=strict)
    File "/Users/macdonald/Desktop/MagicKnobRep/onnx-tensorflow/onnx_tf/handlers/handler.py", line 59, in handle  *
        return ver_handle(node, **kwargs)
    File "/Users/macdonald/Desktop/MagicKnobRep/onnx-tensorflow/onnx_tf/handlers/backend/lstm.py", line 291, in version_14  *
        return cls._common(node, **kwargs)
    File "/Users/macdonald/Desktop/MagicKnobRep/onnx-tensorflow/onnx_tf/handlers/backend/lstm.py", line 249, in _common  *
        outputs, states = cls.rnn(x, tf.compat.v1.nn.rnn_cell.LSTMCell,
    File "/Users/macdonald/Desktop/MagicKnobRep/onnx-t